In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define transformations for training and validation datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'realwaste-main/RealWaste'


In [5]:
# Load the dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms['train'])

# Split dataset into training and validation sets (80:20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply validation transformations to validation dataset
val_dataset.dataset.transform = data_transforms['val']

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Get class names
class_names = full_dataset.classes
print(f"Classes: {class_names}")
print(f"Dataset Size: Train {train_size}, Val: {val_size}")

Classes: ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']
Dataset Size: Train 3801, Val: 951


In [6]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

PyTorch version: 2.2.2
Torchvision version: 0.17.2


In [7]:
from torchvision import models
import torch.nn as nn

# Load the pre-trained ResNet18 model with the correct argument for the weights
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Modify the final fully connected layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# Move the model to the device (CPU or GPU)
model = model.to(device)

In [8]:
import torch.optim as optim

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
def train_model(model, criterion, optimizer, dataloader, dataset_size):
    model.train()  # Set model to training mode
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)  # Calculate the loss

        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects.double() / dataset_size

    print(f'Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
    return epoch_loss, epoch_acc

In [10]:
def validate_model(model, criterion, dataloader, dataset_size):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects.double() / dataset_size

    print(f"Val Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")
    return epoch_loss, epoch_acc


In [14]:
num_epochs = 2  # You can adjust the number of epochs as needed

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Train and validate the model
    train_loss, train_acc = train_model(model, criterion, optimizer, train_loader, train_size)
    val_loss, val_acc = validate_model(model, criterion, val_loader, val_size)

    print("done")

print("Training Complete")


Epoch 1/2
----------
Train Loss: 0.7867, Acc: 0.7324
Val Loss: 1.1596, Acc: 0.6320
done
Epoch 2/2
----------
Train Loss: 0.5970, Acc: 0.7856
Val Loss: 1.1279, Acc: 0.6467
done
Training Complete
